In [6]:
from methods import *
from model import LogisticInstance
import numpy as np
from cvxopt import matrix, solvers
import time

In [19]:
# simple 1D case
# e^{x-2} / (1 + e^{x-2}) (-x + 5) --> max
# theoretical answer: x_opt ~ 2.44
# with constraint x<=2 x_opt ~ 2

a = np.array([1.])
b = -2.
c = np.array([-1.])
d = 5.
F = np.array([[1.]])
g = np.array([2.])

issue = LogisticInstance(a, b, c, d, F, g)

In [20]:
def test(method_name, method, real_f_name, real_f, **kwargs):
    t = time.monotonic()
    x_opt = method(**kwargs)
    print('{}:\n  x_opt = {}\n  {} = {}\n  time = {}s'.format(method_name, x_opt,
                                                       real_f_name, real_f(x_opt),
                                                       time.monotonic() - t))

In [21]:
# unconditional optimization

f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
d2f = lambda x: -issue.d2log_expected_profit(x)



test('Gradient descent, conv. by argument, h_k = 0.1', 
     GradientDescent(h=0.1), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/k', 
     GradientDescent(h=lambda step: 1. / step), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/sqrt(k)', 
     GradientDescent(h=lambda step: step ** -0.5), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

print('\n')

test('Accelerated Nesterov gradient descent, h_k = 0.5', 
     AcceleratedNesterovGradientDescent(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Accelerated Nesterov gradient descent, h_k = 1/sqrt(k)', 
     AcceleratedNesterovGradientDescent(h=lambda step: step ** -0.5),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Broyden-Fletcher-Goldfarb-Shanno method (quasy-Newton)', 
     BroydenFletcherGoldfarbShannoMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.array([1.]), df=df)

Gradient descent, conv. by argument, h_k = 0.1:
  x_opt = [2.44261703]
  expected_profit = 1.5571455818420734
  time = 0.011479351000161842s
Gradient descent, conv. by argument, h_k = 1/k:
  x_opt = [2.34713717]
  expected_profit = 1.554373820931351
  time = 0.1339312929994776s
Gradient descent, conv. by argument, h_k = 1/sqrt(k):
  x_opt = [2.44255797]
  expected_profit = 1.5571455722441265
  time = 0.006747193001501728s


Accelerated Nesterov gradient descent, h_k = 0.5:
  x_opt = [2.4427758]
  expected_profit = 1.5571455971164652
  time = 0.0017869670009531546s
Accelerated Nesterov gradient descent, h_k = 1/sqrt(k):
  x_opt = [2.44126018]
  expected_profit = 1.5571448252478892
  time = 0.0018629240003065206s
Broyden-Fletcher-Goldfarb-Shanno method (quasy-Newton):
  x_opt = [2.4428544]
  expected_profit = 1.557145598997611
  time = 0.0005804309985251166s


In [26]:
# conditional

# the previous issue (theoretically, x_opt=2)

g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('''Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01''', 
     PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g, dg=dg)

# linear
A = matrix(-issue.a)
B = matrix([issue.b])
F = matrix(issue.F)
G = matrix(issue.g)


sol = solvers.lp(A, F, G)
print(sol['x'], issue.prob(np.array(sol['x'])[0]))

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01:
  x_opt = [2.00233491]
  expected_profit = 1.500582363040146
  time = 0.09372364899900276s
Optimal solution found.
[ 2.00e+00]
 0.5


In [28]:
# simple 2d issue
# (x + y) / (1 + e^{x+y}) --> max
# x + y <= 1
#     y <= 0.5
# theoretically, x_opt = (0.5, 0.5)

a = np.array([-1., -1.])
b = 0
c = np.array([1., 1.])
d = 0
F = np.array([[1., 1.],
              [0., 1.]])
g = np.array([1., 0.5])

issue = LogisticInstance(a, b, c, d, F, g)

In [33]:
f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('''Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01''', 
     PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.ones(issue.a.shape[0]) * 0.01, df=df,
     g=g, dg=dg)

# linear
A = matrix(-issue.a)
B = matrix([issue.b])
F = matrix(issue.F)
G = matrix(issue.g)


sol = solvers.lp(A, F, G)
print('  x_opt = 'sol['x'], issue.prob(np.array(sol['x']).T[0]))

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01:
  x_opt = [0.50177973 0.50177973]
  expected_profit = 0.26919696224432765
  time = 0.04589946100168163s
     pcost       dcost       gap    pres   dres   k/t
 0:  1.0000e+00 -2.0000e+00  3e+00  1e+00  3e+00  1e+00
 1: -9.8000e+01 -2.0000e+00  3e+02  1e+00  3e+00  1e+02
 2: -9.9980e+03 -2.0000e+00  3e+04  1e+00  3e+00  1e+04
 3: -1.0000e+06 -2.0000e+00  3e+06  1e+00  3e+00  1e+06
 4: -1.0000e+08 -2.0000e+00  3e+08  1e+00  3e+00  1e+08
Certificate of dual infeasibility found.
[ 5.00e-09]
[-1.00e+00]
 0.7310585786300049


In [16]:
A = matrix(-issue.a)
B = matrix([issue.b])
F = matrix(issue.F)
G = matrix(issue.g)


sol = solvers.lp(A, F, G)
print(sol['x'])

     pcost       dcost       gap    pres   dres   k/t
 0:  1.0000e+00 -2.0000e+00  3e+00  1e+00  3e+00  1e+00
 1: -9.8000e+01 -2.0000e+00  3e+02  1e+00  3e+00  1e+02
 2: -9.9980e+03 -2.0000e+00  3e+04  1e+00  3e+00  1e+04
 3: -1.0000e+06 -2.0000e+00  3e+06  1e+00  3e+00  1e+06
 4: -1.0000e+08 -2.0000e+00  3e+08  1e+00  3e+00  1e+08
Certificate of dual infeasibility found.
[ 5.00e-09]
[-1.00e+00]

